# Managing Analyzers in Your Resource

This notebook demonstrates how to create a simple analyzer and manage its lifecycle.

## Prerequisites
1. Ensure your Azure AI service is configured following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [ ]:
%pip install -r ../requirements.txt

## Create the Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class that provides functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, this client serves as a lightweight SDK.

> Fill the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with your Azure AI Service details.

> ⚠️ Important:
Update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample might not run correctly.

> ⚠️ Note: Using a subscription key works, but using Azure Active Directory (AAD) token-based authentication is more secure and highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key. Use only one of these methods.
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set it in your ".env" file if not using token auth
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

# Add the parent directory to the system path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this line if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/analyzer_management",  # This header is used for sample usage telemetry. Please comment out this line if you want to opt out.
)

## Create a Simple Analyzer
First, we create an analyzer from a template to extract invoice fields.

In [ ]:
import uuid

ANALYZER_TEMPLATE = "../analyzer_templates/call_recording_analytics.json"
CUSTOM_ANALYZER_ID = "analyzer-management-sample-" + str(uuid.uuid4())

response = client.begin_create_analyzer(CUSTOM_ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

## List All Analyzers in Your Resource

After successfully creating an analyzer, you can use it to analyze our input files. You can also list all analyzers available in your resource.

In [ ]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"Details of the first 3 analyzers: {json.dumps(response['value'][:3], indent=2)}")
print(f"Details of the last analyzer: {json.dumps(response['value'][-1], indent=2)}")

## Get Analyzer Details by ID

Keep track of the analyzer ID when you create it. Use the ID to retrieve detailed analyzer definitions later.

In [ ]:
result_json = client.get_analyzer_detail_by_id(CUSTOM_ANALYZER_ID)
print(json.dumps(result_json, indent=2))

## Delete an Analyzer
If you no longer need an analyzer, delete it using its ID.

In [ ]:
client.delete_analyzer(CUSTOM_ANALYZER_ID)